# COMBINING DATA

USE ONLY WHEN NEEDED TO COMBINE THE FILES

In [29]:
import pandas as pd
import glob

# # Get a list of all Excel files with the specified naming pattern
# dc_files = glob.glob("dc-main-series_*.xlsx")
# image_files = glob.glob("image-main-series_*.xlsx")

# # Initialize an empty DataFrame to store merged data
# merged_data = pd.DataFrame()

# for file in dc_files:
#     df = pd.read_excel(file)
#     merged_data = merged_data.append(df, ignore_index=True)

# for file in image_files:
#     df = pd.read_excel(file)
#     merged_data = merged_data.append(df, ignore_index=True)
# merged_data.to_excel("merged_file.xlsx", index=False)


In [30]:
data = pd.read_excel("merged_file.xlsx")
data

,series_title,issue_number,issue_comments,issue_edition,published_date,comic_era,issue_image,issue_image_url
0,Batman: Battle For The Cowl - The Underground,#1,NaN,NaN,2009-06-01 00:00:00,Modern Age,https://comicspriceguide.com/image/ulstm/wowvi...,https://comicspriceguide.com/image/ulstm/wowvix
1,Flashpoint: Reverse-Flash,#1,NaN,NaN,2011-08-01 00:00:00,Modern Age,https://comicspriceguide.com/image/tluvh/yhwth...,https://comicspriceguide.com/image/tluvh/yhwthw
2,Green Lantern / Sentinel: Heart Of Darkness,#1,NaN,NaN,1998-03-01 00:00:00,Modern Age,https://comicspriceguide.com/image/pivvh/pbxwn...,https://comicspriceguide.com/image/pivvh/pbxwnw
3,Green Lantern / Sentinel: Heart Of Darkness,#2,NaN,NaN,1998-04-01 00:00:00,Modern Age,https://comicspriceguide.com/image/pivvh/pbxwn...,https://comicspriceguide.com/image/pivvh/pbxwnv
4,Green Lantern / Sentinel: Heart Of Darkness,#3,NaN,NaN,1998-05-01 00:00:00,Modern Age,https://comicspriceguide.com/image/pivvh/pbxwn...,https://comicspriceguide.com/image/pivvh/pbxwny
...,...,...,...,...,...,...,...,...
59858,Fishflies (Fcbd 2023),#1,NaN,NaN,2023-Current,Modern Age,https://comicspriceguide.com/image/phuwby/piqx...,https://comicspriceguide.com/image/phuwby/piqxiqs
59859,Ophiuchus,#1,NaN,NaN,2019,Modern Age,https://comicspriceguide.com/image/pkspip/pkwvcxp,https://comicspriceguide.com/image/pkspip/pkwvcxp
59860,Street Angel Vs Ninjatech,#1,NaN,NaN,2018,Modern Age,https://comicspriceguide.com/image/pjxxny/pkutiqq,https://comicspriceguide.com/image/pjxxny/pkutiqq
59861,Me The People,#1,NaN,NaN,2018,Modern Age,https://comicspriceguide.com/image/pjxwmq/pkuriuu,https://comicspriceguide.com/image/pjxwmq/pkuriuu


In [31]:
data.isna().sum()

series_title           0
issue_number           0
issue_comments     39426
issue_edition      45210
published_date         0
comic_era              0
issue_image            0
issue_image_url        0
dtype: int64

In [32]:
data.describe()

,series_title,issue_number,issue_comments,issue_edition,published_date,comic_era,issue_image,issue_image_url
count,59863,59863,20437,14653,59863,59863,59863,59863
unique,2895,5512,12008,3,1506,5,59304,59304
top,Batman,#1,Polybagged Digital Combo Pack Variant,DIRECT EDITION,1952-1977,Modern Age,https://comicspriceguide.com/img/missing_thm.jpg,https://comicspriceguide.com/img/missing_lrg.jpg
freq,2089,2916,286,8252,240,39143,477,477


In [33]:
duplicate_rows = data[data.duplicated()]
data = data.drop_duplicates()

# CLEAN IMAGE URLs

In [34]:
# image_urls = data['issue_image_url']
# data['image_download_error'] = False
# for i, url in enumerate(image_urls):
#     try:
#         req = urllib.request.urlopen(url)
#         arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
#         img = cv2.imdecode(arr, -1)
#     except:
#         data.loc[i, 'image_download_error'] = True
# data = data[data['image_download_error'] == False].drop(columns='image_download_error')

# UPDATED

In [35]:
import pandas as pd
import urllib.request
import numpy as np
import cv2
import os
from sklearn.feature_extraction import image
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
import tensorflow_hub as hub

In [36]:
# Load the CSV file
excel_file_path = "merged_file.xlsx"
data = pd.read_excel(excel_file_path)

# Get the image URLs and details from the Excel
image_urls = data['issue_image_url']
image_attributes = data[['series_title', 'issue_number', 'issue_comments', 'issue_edition', 'published_date', 'comic_era', 'issue_image', 'issue_image_url']]

In [38]:
def preprocess_images(image_urls, image_attributes, save_directory):
    images = []
    filenames = []
    image_paths = []

    dropped_rows = []

    for i, url in enumerate(image_urls):
        try:
            req = urllib.request.urlopen(url)
            arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
            img = cv2.imdecode(arr, -1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            filename = str(i+1)
            filenames.append(filename)
            image_path = os.path.join(save_directory, f"preprocessed_{filename}.jpg")
            cv2.imwrite(image_path, img)
            image_paths.append(image_path)
            print(f"Downloaded and processed image {i+1} of {len(image_urls)}")
        except:
            print(f"Error downloading image {i+1}")
            dropped_rows.append(i)

    # Drop rows with image download errors
    image_urls = image_urls.drop(dropped_rows)
    image_attributes = image_attributes.drop(dropped_rows)

    # Create a DataFrame with image details
    image_details = pd.DataFrame({
        'Filename': filenames,
        'Image_Path': image_paths
    })
    image_details = pd.concat([image_details, image_attributes], axis=1)

    # Save image details to a CSV file
    csv_path = os.path.join(save_directory, 'Final.csv')
    image_details.to_csv(csv_path, index=False)
    print(f"Preprocessed image details saved to: {csv_path}")

    return images, filenames, image_paths, image_urls, image_attributes


In [39]:
# Specify the directory to save the preprocessed images and CSV file
save_directory = "Final"

# Create the save directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Download and preprocess the database images, and save details to CSV
images, filenames, image_paths, image_urls, image_attributes = preprocess_images(image_urls, image_attributes, save_directory)

Downloaded and processed image 1 of 10
Downloaded and processed image 2 of 10
Downloaded and processed image 3 of 10
Downloaded and processed image 4 of 10
Downloaded and processed image 5 of 10
Downloaded and processed image 6 of 10
Downloaded and processed image 7 of 10
Downloaded and processed image 8 of 10
Downloaded and processed image 9 of 10
Downloaded and processed image 10 of 10
Preprocessed image details saved to: Final/Final.csv


## TO CONVERT TO ZIP

In [40]:
import zipfile
import os

def create_zip(folder_path, zip_path):
    # Create a ZIP file
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Iterate over all files and subdirectories in the folder
        for root, _, files in os.walk(folder_path):
            for file in files:
                # Get the full path of the file
                file_path = os.path.join(root, file)

                # Add the file to the ZIP
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Specify the folder path and ZIP file path
folder_path = '/content/Final'  # Replace with your folder path
zip_path = '/content/Final.zip'  # Replace with the desired ZIP file path

# Create the ZIP file
create_zip(folder_path, zip_path)


In [11]:
# def load_images_from_folder(folder_path):
#     images = []
#     for filename in os.listdir(folder_path):
#         if filename.startswith("preprocessed_"):
#             img = cv2.imread(os.path.join(folder_path, filename))
#             if img is not None:
#                 images.append(img)
#                 print(f"Loaded_{filename}")
#     return images

# # Folder path where the preprocessed images are stored
# folder_path = "Final"

# # Load the preprocessed images from the folder
# images = load_images_from_folder(folder_path)

Loaded_preprocessed_7.jpg
Loaded_preprocessed_5.jpg
Loaded_preprocessed_4.jpg
Loaded_preprocessed_1.jpg
Loaded_preprocessed_10.jpg
Loaded_preprocessed_3.jpg
Loaded_preprocessed_6.jpg
Loaded_preprocessed_9.jpg
Loaded_preprocessed_8.jpg
Loaded_preprocessed_2.jpg


In [ ]:
# images[0]

In [41]:
def find_similar_images(query_image_path, images, filenames, attributes, k=1):
    query_image = cv2.imread(query_image_path)
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image = cv2.resize(query_image, (224, 224))
    query_image = np.reshape(query_image, (1, -1))

    reshaped_images = np.reshape(images, (len(images), -1))

    nbrs = NearestNeighbors(n_neighbors=k).fit(reshaped_images)

    # Save the NearestNeighbors model
    joblib.dump(nbrs, 'nearest_neighbors_model.pkl')
    # Load the NearestNeighbors model
    nbrs = joblib.load('nearest_neighbors_model.pkl')


    distances, indices = nbrs.kneighbors(query_image)

    for i in range(k):
        index = indices[0][i]
        filename = filenames[index]
        matched_attributes = attributes.iloc[index]

        print(f"Similar Image {i+1}: {filename}")
        print("Attributes:")
        for attr_name, attr_value in matched_attributes.iteritems():
            print(f"{attr_name}: {attr_value}")
        print()

# data = pd.read_csv('Final.csv')
# filenames = data['Filename']
# image_attributes = data[['series_title', 'issue_number', 'issue_comments', 'issue_edition', 'published_date', 'comic_era', 'issue_image', 'issue_image_url']]

# Find similar images and display attributes
find_similar_images("img.jpg", images, filenames, image_attributes)


Similar Image 1: 1
Attributes:
series_title: Batman: Battle For The Cowl - The Underground
issue_number: #1
issue_comments: nan
issue_edition: nan
published_date: 2009-06-01 00:00:00
comic_era: Modern Age
issue_image: https://comicspriceguide.com/image/ulstm/wowvix/thm
issue_image_url: https://comicspriceguide.com/image/ulstm/wowvix



<ipython-input-41-f6858ee4b65b>:26: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for attr_name, attr_value in matched_attributes.iteritems():


In [42]:
import joblib

def Load_model(images, k=1):

    reshaped_images = np.reshape(images, (len(images), -1))
    nbrs = NearestNeighbors(n_neighbors=k).fit(reshaped_images)
    joblib.dump(nbrs, 'nearest_neighbors_model.pkl')

Load_model(images)

In [56]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.neighbors import NearestNeighbors


def find_similar_images(query_image_path, images, filenames, attributes, k=1):
    query_image = cv2.imread(query_image_path)
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image = cv2.resize(query_image, (224, 224))
    query_image = np.reshape(query_image, (1, -1))

    nbrs = joblib.load('nearest_neighbors_model.pkl')
    distances, indices = nbrs.kneighbors(query_image)
    print(distances)


    for i in range(k):
        index = indices[0][i]
        filename = filenames[index]
        matched_attributes = attributes.iloc[index]

        print(f"Similar Image: {filename}")
        print("Attributes:")
        for attr_name, attr_value in matched_attributes.iteritems():
            print(f"{attr_name}: {attr_value}")
        print()

data = pd.read_csv('Final.csv')
filenames = data['Filename']
image_attributes = data[['series_title', 'issue_number', 'issue_comments', 'issue_edition', 'published_date', 'comic_era', 'issue_image', 'issue_image_url']]

# Find similar images and display attributes
find_similar_images("img.jpg", images, filenames, image_attributes)


[[0.]]
Similar Image: 1
Attributes:
series_title: Batman: Battle For The Cowl - The Underground
issue_number: #1
issue_comments: nan
issue_edition: nan
published_date: 2009-06-01 00:00:00
comic_era: Modern Age
issue_image: https://comicspriceguide.com/image/ulstm/wowvix/thm
issue_image_url: https://comicspriceguide.com/image/ulstm/wowvix



<ipython-input-56-e600833f181a>:26: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for attr_name, attr_value in matched_attributes.iteritems():


In [62]:
def find_similar_images(query_image_path, images, filenames, attributes, k=1, similarity_threshold=0.9):
    query_image = cv2.imread(query_image_path)
    query_image = cv2.cvtColor(query_image, cv2.COLOR_BGR2RGB)
    query_image = cv2.resize(query_image, (224, 224))
    query_image = np.reshape(query_image, (1, -1))

    nbrs = joblib.load('nearest_neighbors_model.pkl')
    distances, indices = nbrs.kneighbors(query_image)

    for i in range(k):
        distance = distances[0][i]
        if distance >= similarity_threshold:
            print("If Running")
            index = indices[0][i]
            filename = filenames[index]
            matched_attributes = attributes.iloc[index]

            print(f"Similar Image: {filename}")
            print("Attributes:")
            for attr_name, attr_value in matched_attributes.iteritems():
                print(f"{attr_name}: {attr_value}")
            print()

        elif distances == [[0.]]:
            print("Elseif Running")
            index = indices[0][i]
            filename = filenames[index]
            matched_attributes = attributes.iloc[index]

            print(f"Similar Image: {filename}")
            print("Attributes:")
            for attr_name, attr_value in matched_attributes.iteritems():
                print(f"{attr_name}: {attr_value}")
            print()
        else:
            print("Else Running")
            print("No nearest image found.")

data = pd.read_csv('Final.csv')
filenames = data['Filename']
image_attributes = data[['series_title', 'issue_number', 'issue_comments', 'issue_edition', 'published_date', 'comic_era', 'issue_image', 'issue_image_url']]

# Find similar images and display attributes
find_similar_images("imgtest.jpeg", images, filenames, image_attributes, k=1, similarity_threshold=0.9)


If Running
Similar Image: 1
Attributes:
series_title: Batman: Battle For The Cowl - The Underground
issue_number: #1
issue_comments: nan
issue_edition: nan
published_date: 2009-06-01 00:00:00
comic_era: Modern Age
issue_image: https://comicspriceguide.com/image/ulstm/wowvix/thm
issue_image_url: https://comicspriceguide.com/image/ulstm/wowvix



<ipython-input-62-e5fc0ccd2d8e>:20: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for attr_name, attr_value in matched_attributes.iteritems():
